<a href="https://colab.research.google.com/github/UkeshThapa/Human-Activities-recognition/blob/main/Research_Human_activities_Recognition_TML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Research on Human Activities Recognition
#### @author : John Bernardo
#### @email : jaybz.bernardo@gmail.com


#### Code By:
#### @author : Ukesh Thapa
#### @email : yukeshthapa8@gmail.com

#### Requirements :
#### Load Dataset
#### Split dataset
#### Use Different Model
#### Hypertune Parameter
#### Evaluation Metrics: AUC - ROC, Confusion Metrics
#### Cross Validation - K fold cross validation